In [1]:
# importing required libraries

import re
import mlflow
import string
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
# reading csv file

df = pd.read_csv('IMDB.csv')

In [3]:
# quick look at data

df.head()

,review,sentiment
0,Film version of Sandra Bernhard's one-woman of...,negative
1,I switched this on (from cable) on a whim and ...,positive
2,The `plot' of this film contains a few holes y...,negative
3,"Some amusing humor, some that falls flat, some...",negative
4,What can you say about this movie? It was not ...,negative


In [4]:
# data preprocessing

# text preprocessing functions
def lemmatization(text: str):
    """
    Lemmatize the text
    """
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word= word) for word in text]
    return " ".join(text)

def remove_stop_words(text: str):
    """
    Remove stop words from the text
    """
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text: str):
    """
    Removes numbers from the text
    """
    text = "".join([char for char in text if not char.isdigit()])
    return text

def lower_case(text: str):
    """
    Convert text to lower case
    """
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text: str):
    """
    Remove punctuations from the text
    """
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace(':', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text: str):
    """
    Remove URLs from the text
    """
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df: pd.DataFrame):
    """
    Normalize the text data
    """
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(lower_case)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
# applying text preprocessing
df = normalize_text(df)

# quick look after normalising
df.head()

,review,sentiment
0,film version of sandra bernhard's one-woman of...,negative
1,i switched this on (from cable) on a whim and ...,positive
2,the `plot' of this film contains a few holes y...,negative
3,"some amusing humor, some that falls flat, some...",negative
4,what can you say about this movie? it was not ...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    517
positive    483
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,film version of sandra bernhard's one-woman of...,0
1,i switched this on (from cable) on a whim and ...,1
2,the `plot' of this film contains a few holes y...,0
3,"some amusing humor, some that falls flat, some...",0
4,what can you say about this movie? it was not ...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/ashishsoni295work/Sentiment-Analysis.mlflow')
dagshub.init(repo_owner='ashishsoni295work', repo_name='Sentiment-Analysis', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

Accessing as ashishsoni295work

Initialized MLflow to track repo "ashishsoni295work/Sentiment-Analysis"

Repository ashishsoni295work/Sentiment-Analysis initialized!

2025/12/18 22:30:46 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/6a5745ccec1849bc90c9e6bb108ec546', creation_time=1766077247029, experiment_id='0', last_update_time=1766077247029, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
import logging
import os
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-12-18 22:31:11,147 - INFO - Starting MLflow run...
2025-12-18 22:31:11,600 - INFO - Logging preprocessing parameters...
2025-12-18 22:31:12,830 - INFO - Initializing Logistic Regression model...
2025-12-18 22:31:12,830 - INFO - Fitting the model...
2025-12-18 22:31:12,873 - INFO - Model training complete.
2025-12-18 22:31:12,874 - INFO - Logging model parameters...
2025-12-18 22:31:13,271 - INFO - Making predictions...
2025-12-18 22:31:13,272 - INFO - Calculating evaluation metrics...
2025-12-18 22:31:13,279 - INFO - Logging evaluation metrics...
2025-12-18 22:31:14,981 - INFO - Saving and logging the model...
2025/12/18 22:31:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-18 22:31:25,609 - INFO - Model training and logging completed in 14.01 seconds.
2025-12-18 22:31:25,610 - INFO - Accuracy: 0.648
2025-12-18 22:31:25,610 - INFO - Precision: 0.6343283582089553
2025-12-18 22:31:25,610 - INFO - Recall: 0.6854838709677419
2025-12-18

🏃 View run selective-kit-833 at: https://dagshub.com/ashishsoni295work/Sentiment-Analysis.mlflow/#/experiments/0/runs/9de7e783f2174de19bc2c85e8d194cc0
🧪 View experiment at: https://dagshub.com/ashishsoni295work/Sentiment-Analysis.mlflow/#/experiments/0
